<a href="https://colab.research.google.com/github/Victorlouisdg/simulators/blob/main/cloth_bw_bending_2nd_derivatives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
def skew(v):
    return np.array([[0, -v[2], v[1]],
                     [v[2], 0, -v[0]],
                     [-v[1], v[0], 0]])

In [3]:
nA_xx = {}
nB_xx = {}

# The 2 arrays below can be index in order [m][n]
# each entry here should be multiplied by a 3x3 identity matrix
dqA_dx = np.array([
    (0, -1, 1, 0),
    (1, 0, -1, 0),
    (-1, 1, 0, 0),
    (0, 0, 0, 0)    
])

dqB_dx = np.array([
    (0, 0, 0, 0),
    (0, 0, 1, -1),
    (0, -1, 0, 1),
    (0, 1, -1, 0)
])

for m in range(4):
    nA_xx[m] = {}
    nB_xx[m] = {}
    for n in range(4):
        nA_xx[m][n] = {}
        nB_xx[m][n] = {}
        for s in range(3):
            nA_xx[m][n][s] = {}
            nB_xx[m][n][s] = {}
            for t in range(3):
                dqA_dx_mnt = np.zeros(3)
                dqA_dx_mnt[t] = dqA_dx[m][n]
                SA = skew(dqA_dx_mnt)

                dqB_dx_mnt = np.zeros(3)
                dqB_dx_mnt[t] = dqB_dx[m][n]
                SB = skew(dqB_dx_mnt)
                
                nA_xx[m][n][s][t] = SA[s, :]
                nB_xx[m][n][s][t] = SB[s, :]

                # These to lines replace -0.0 with 0.0
                nA_xx[m][n][s][t] = np.where(nA_xx[m][n][s][t]==0.0, 0.0, nA_xx[m][n][s][t])
                nB_xx[m][n][s][t] = np.where(nB_xx[m][n][s][t]==0.0, 0.0, nB_xx[m][n][s][t])



In [4]:
total_nA_components = 4 * 4 * 3 * 3 * 3
nonzero_nA_components = 0

for m in range(4):
    for n in range(4):
        for s in range(3):
            for t in range(3):
                print(m, n, s, t, nA_xx[m][n][s][t], nB_xx[m][n][s][t])
                for c in nA_xx[m][n][s][t]:
                    if not np.isclose(c, 0.0):
                        nonzero_nA_components += 1

print("nonzero_nA_components: ", nonzero_nA_components)
print("total_nA_components: ", total_nA_components)

print("fill percentage: ", nonzero_nA_components / total_nA_components)

0 0 0 0 [0. 0. 0.] [0. 0. 0.]
0 0 0 1 [0. 0. 0.] [0. 0. 0.]
0 0 0 2 [0. 0. 0.] [0. 0. 0.]
0 0 1 0 [0. 0. 0.] [0. 0. 0.]
0 0 1 1 [0. 0. 0.] [0. 0. 0.]
0 0 1 2 [0. 0. 0.] [0. 0. 0.]
0 0 2 0 [0. 0. 0.] [0. 0. 0.]
0 0 2 1 [0. 0. 0.] [0. 0. 0.]
0 0 2 2 [0. 0. 0.] [0. 0. 0.]
0 1 0 0 [0. 0. 0.] [0. 0. 0.]
0 1 0 1 [ 0.  0. -1.] [0. 0. 0.]
0 1 0 2 [0. 1. 0.] [0. 0. 0.]
0 1 1 0 [0. 0. 1.] [0. 0. 0.]
0 1 1 1 [0. 0. 0.] [0. 0. 0.]
0 1 1 2 [-1.  0.  0.] [0. 0. 0.]
0 1 2 0 [ 0. -1.  0.] [0. 0. 0.]
0 1 2 1 [1. 0. 0.] [0. 0. 0.]
0 1 2 2 [0. 0. 0.] [0. 0. 0.]
0 2 0 0 [0. 0. 0.] [0. 0. 0.]
0 2 0 1 [0. 0. 1.] [0. 0. 0.]
0 2 0 2 [ 0. -1.  0.] [0. 0. 0.]
0 2 1 0 [ 0.  0. -1.] [0. 0. 0.]
0 2 1 1 [0. 0. 0.] [0. 0. 0.]
0 2 1 2 [1. 0. 0.] [0. 0. 0.]
0 2 2 0 [0. 1. 0.] [0. 0. 0.]
0 2 2 1 [-1.  0.  0.] [0. 0. 0.]
0 2 2 2 [0. 0. 0.] [0. 0. 0.]
0 3 0 0 [0. 0. 0.] [0. 0. 0.]
0 3 0 1 [0. 0. 0.] [0. 0. 0.]
0 3 0 2 [0. 0. 0.] [0. 0. 0.]
0 3 1 0 [0. 0. 0.] [0. 0. 0.]
0 3 1 1 [0. 0. 0.] [0. 0. 0.]
0 3 1 2 [0. 0. 0.] [0.

In [5]:
print("const float3 normalA_second_derivatives[4][4][3][3] = {")
for m in range(4):
    print("{")
    for n in range(4):
        print("{")
        for s in range(3):
            print("{")
            print("/* m = " + str(m) + ", n = " + str(n) + ", s = " + str(s) + " */")
            for t in range(3):
                # print(m, n, s, t, nA_xx[m][n][s][t], nB_xx[m][n][s][t])
                x, y, z = nA_xx[m][n][s][t]
                print("float3(" + str(x) + "f, " + str(y) + "f, " + str(z) + "f), ")
            print("},")
        print("},")
    print("},")
print("};")

const float3 normalA_second_derivatives[4][4][3][3] = {
{
{
{
/* m = 0, n = 0, s = 0 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 0, s = 1 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 0, s = 2 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
},
{
{
/* m = 0, n = 1, s = 0 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, -1.0f), 
float3(0.0f, 1.0f, 0.0f), 
},
{
/* m = 0, n = 1, s = 1 */
float3(0.0f, 0.0f, 1.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(-1.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 1, s = 2 */
float3(0.0f, -1.0f, 0.0f), 
float3(1.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
},
{
{
/* m = 0, n = 2, s = 0 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 1.0f), 
float3(0.0f, -1.0f, 0.0f), 
},
{
/* m = 0, n = 2, s = 1 */
float3(0.0f, 0.0f, -1.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(1.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 2, s = 2 */
f

In [6]:
print("const float3 normalB_second_derivatives[4][4][3][3] = {")
for m in range(4):
    print("{")
    for n in range(4):
        print("{")
        for s in range(3):
            print("{")
            print("/* m = " + str(m) + ", n = " + str(n) + ", s = " + str(s) + " */")
            for t in range(3):
                # print(m, n, s, t, nA_xx[m][n][s][t], nB_xx[m][n][s][t])
                x, y, z = nB_xx[m][n][s][t]
                print("float3(" + str(x) + "f, " + str(y) + "f, " + str(z) + "f), ")
            print("},")
        print("},")
    print("},")
print("};")

const float3 normalB_second_derivatives[4][4][3][3] = {
{
{
{
/* m = 0, n = 0, s = 0 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 0, s = 1 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 0, s = 2 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
},
{
{
/* m = 0, n = 1, s = 0 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 1, s = 1 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 1, s = 2 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
},
{
{
/* m = 0, n = 2, s = 0 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 2, s = 1 */
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
float3(0.0f, 0.0f, 0.0f), 
},
{
/* m = 0, n = 2, s = 2 */
float3